# RandomForest Classifier Implementation with Pipelines and Hyperparameter Tuning

In [1]:
import seaborn as sns

In [2]:
df=sns.load_dataset('tips')

In [3]:
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [4]:
df.isnull().sum()

total_bill    0
tip           0
sex           0
smoker        0
day           0
time          0
size          0
dtype: int64

In [5]:
df['time'].unique()

['Dinner', 'Lunch']
Categories (2, object): ['Lunch', 'Dinner']

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    category
 6   size        244 non-null    int64   
dtypes: category(4), float64(2), int64(1)
memory usage: 7.4 KB


In [7]:
from sklearn.preprocessing import LabelEncoder

In [8]:
encoder=LabelEncoder()

In [9]:
df['time']=encoder.fit_transform(df['time'])

In [10]:
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,0,2
1,10.34,1.66,Male,No,Sun,0,3
2,21.01,3.50,Male,No,Sun,0,3
3,23.68,3.31,Male,No,Sun,0,2
4,24.59,3.61,Female,No,Sun,0,4


In [11]:
df['time'].value_counts()

time
0    176
1     68
Name: count, dtype: int64

In [12]:
## Independent and Dependent Features
X=df.drop(labels=['time'],axis=1)
y=df['time']

In [13]:
X.head()

,total_bill,tip,sex,smoker,day,size
0,16.99,1.01,Female,No,Sun,2
1,10.34,1.66,Male,No,Sun,3
2,21.01,3.50,Male,No,Sun,3
3,23.68,3.31,Male,No,Sun,2
4,24.59,3.61,Female,No,Sun,4


In [14]:
y.head()

0    0
1    0
2    0
3    0
4    0
Name: time, dtype: int32

In [15]:
from sklearn.model_selection import train_test_split

In [16]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [17]:
#Handling Missing Values
from sklearn.impute import SimpleImputer

#Handling Categorical Features
from sklearn.preprocessing import OneHotEncoder

#Feature Scaling
from sklearn.preprocessing import StandardScaler

#Automating the Entire Process
from sklearn.pipeline import Pipeline

#Column Tranform
from sklearn.compose import ColumnTransformer

In [18]:
categorical_cols=['sex','smoker','day']
numerical_cols=['total_bill','tip','size']

### Feature Engineering Automation

In [19]:
num_pipeline=Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='median')), # Handling Missing Values
        ('scaler',StandardScaler()) # Feature Scaling
    ]
)

In [20]:
categorical_pipeline=Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='most_frequent')), # Handling Missing Values
        ('oneHotEncoder',OneHotEncoder()) # Categorical Features to Numerical
    ]
)

In [21]:
preprocessor=ColumnTransformer([
    ('num_pipeline',num_pipeline,numerical_cols),
    ('categorical_pipeline',categorical_pipeline,categorical_cols)
])

In [22]:
X_train=preprocessor.fit_transform(X_train)
X_test=preprocessor.transform(X_test)

In [32]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

### Automate the Model Training Process

In [33]:
models={
    'Random Forest':RandomForestClassifier(),
    'Decision Tree':DecisionTreeClassifier(),
    'Support Vector Classifier':SVC()
}

In [25]:
from sklearn.metrics import accuracy_score

In [34]:
def evaluate_model(X_train, y_train, X_test, y_test, models):
    results = {}
    for model_name, model in models.items():

        model.fit(X_train, y_train)  # Train the model
        
        y_test_pred = model.predict(X_test)  # Predict the test set

        # Calculate and store the metrics
        results[model_name] = {
            "accuracy": accuracy_score(y_test, y_test_pred)
        }
    return results

In [35]:
# Call the evaluate_model function with debug prints
results = evaluate_model(X_train, y_train, X_test, y_test, models)
print(results)

{'Random Forest': {'accuracy': 0.9591836734693877}, 'Decision Tree': {'accuracy': 0.9387755102040817}, 'Support Vector Classifier': {'accuracy': 0.9591836734693877}}


In [36]:
## Hyperparameter Tuning for RandomForestClassifier
params={
    'max_depth':[3,5,10,None],
    'n_estimators':[100,200,300],
    'criterion':['gini','entropy']
}

In [37]:
from sklearn.model_selection import RandomizedSearchCV

In [38]:
cv=RandomizedSearchCV(RandomForestClassifier(),param_distributions=params,cv=5,scoring='accuracy',verbose=3)

In [39]:
cv.fit(X_train,y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END criterion=entropy, max_depth=10, n_estimators=200;, score=0.974 total time=   0.1s
[CV 2/5] END criterion=entropy, max_depth=10, n_estimators=200;, score=0.923 total time=   0.1s
[CV 3/5] END criterion=entropy, max_depth=10, n_estimators=200;, score=1.000 total time=   0.1s
[CV 4/5] END criterion=entropy, max_depth=10, n_estimators=200;, score=0.923 total time=   0.1s
[CV 5/5] END criterion=entropy, max_depth=10, n_estimators=200;, score=0.923 total time=   0.1s
[CV 1/5] END criterion=gini, max_depth=10, n_estimators=300;, score=0.974 total time=   0.1s
[CV 2/5] END criterion=gini, max_depth=10, n_estimators=300;, score=0.923 total time=   0.1s
[CV 3/5] END criterion=gini, max_depth=10, n_estimators=300;, score=1.000 total time=   0.1s
[CV 4/5] END criterion=gini, max_depth=10, n_estimators=300;, score=0.949 total time=   0.1s
[CV 5/5] END criterion=gini, max_depth=10, n_estimators=300;, score=0.923 total time=  

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(),
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': [3, 5, 10, None],
                                        'n_estimators': [100, 200, 300]},
                   scoring='accuracy', verbose=3)

In [40]:
cv.best_params_

{'n_estimators': 300, 'max_depth': 10, 'criterion': 'gini'}

In [41]:
classifier=RandomForestClassifier(n_estimators=300,max_depth=10,criterion='gini')

In [42]:
classifier.fit(X_train,y_train)

RandomForestClassifier(max_depth=10, n_estimators=300)

In [43]:
y_pred_classifier=classifier.predict(X_test)

In [44]:
# Accuracy using Hyperparameter Tuning
accuracy_score(y_test,y_pred_classifier)

0.9591836734693877